In [ ]:
import openai
import tqdm
import time
import json
import shortuuid


client = openai.OpenAI(api_key=snakemake.params.openai_api_key)  # Initialize the OpenAI client

def gpt4_response(question, max_tokens: int = 1024, **kwargs):
    messages = [
        {
        "role": "user",
        "content": question
    }]
    #print(messages)
    for i in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.0,
            )
            return response.choices[0].message.content
        except Exception as e:
            print(e)
    else:
        raise RuntimeError()

In [ ]:
with open(snakemake.input.transcriptome_text_features) as f:
    question_informations = [json.loads(line) for line in f.readlines()]

In [ ]:
question_informations[0]

In [ ]:
responses = [gpt4_response(info["text"]) for info in tqdm.tqdm(question_informations)]


In [ ]:
responses[0]

In [ ]:
with open(snakemake.output[0], "w") as f:
    for response, info in zip(responses, question_informations):
        json.dump({
            "question_id": info["question_id"],
            "text": response,
            "answer_id": shortuuid.uuid(),
            "model_id": "gpt-4-turbo-preview", 
            "metadata": {}
        }, f)
        f.write("\n")

In [ ]:
response